In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime
now = datetime.now()
print(now)


2023-01-10 14:16:16.081591


#### Connect to Sharepoint Microsoft

In [2]:
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext


def get_sharepoint_context_using_user():

    # Get sharepoint credentials
    sharepoint_url = 'https://tanthanhdat.sharepoint.com/sites/THNGTINCHUNG/Shared%20Documents/Forms/AllItems.aspx?id=%2Fsites%2FTHNGTINCHUNG%2FShared%20Documents%2FGeneral%2F16%2E%20DATA%20ANALYSYS&p=true&ga=1'

    # Initialize the client credentials
    user_credentials = UserCredential('data-analyst@Tanthanhdat.onmicrosoft.com', 'Truongvietbao123')

    # create client context object
    ctx = ClientContext(sharepoint_url).with_credentials(user_credentials)

    return ctx

In [9]:
def upload_to_sharepoint(dir_name: str, file_name: str):

    sp_relative_url = create_sharepoint_directory(dir_name)
    ctx = get_sharepoint_context()

    target_folder = ctx.web.get_folder_by_server_relative_url(sp_relative_url)

    with open(file_name, 'rb') as content_file:
        file_content = content_file.read()
        target_folder.upload_file(file_name, file_content).execute_query()

In [10]:
from urllib import response
import environ
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.files.file import File
import datetime

env = environ.Env()
environ.Env.read_env()

USERNAME = env('sharepoint_email')
PASSWORD = env('sharepoint_password')
SHAREPOINT_SITE = env('sharepoint_url_site')
SHAREPOINT_SITE_NAME = env('sharepoint_site_name')
SHAREPOINT_DOC = env('sharepoint_doc_library')

class SharePoint:
    def _auth(self):
        conn = ClientContext(SHAREPOINT_SITE).with_credentials(
            UserCredential(
                USERNAME,
                PASSWORD
            )
        )
        return conn

    def _get_files_list(self, folder_name):
        conn = self._auth()
        target_folder_url = f'{SHAREPOINT_DOC}/{folder_name}'
        root_folder = conn.web.get_folder_by_server_relative_url(target_folder_url)
        root_folder.expand(["Files", "Folders"]).get().execute_query()
        return root_folder.files

    def download_file(self, file_name, folder_name):
        conn = self._auth()
        file_url = f'/sites/{SHAREPOINT_SITE_NAME}/{SHAREPOINT_DOC}/{folder_name}/{file_name}'
        file = File.open_binary(conn, file_url)
        return file.content
    
    def download_latest_file(self, folder_name):
        date_format = "%Y-%m-%dT%H:%M:%SZ"
        files_list = self._get_files_list(folder_name)
        file_dict = {}
        for file in files_list:
            dt_obj = datetime.datetime.strptime(file.time_last_modified, date_format)
            file_dict[file.name] = dt_obj
        # sort dict object to get the latest file
        file_dict_sorted = {key:value for key, value in sorted(file_dict.items(), key=lambda item:item[1], reverse=True)}    
        latest_file_name = next(iter(file_dict_sorted))
        content = self.download_file(latest_file_name, folder_name)
        return latest_file_name, content
        

    def upload_file(self, file_name, folder_name, content):
        conn = self._auth()
        target_folder_url = f'/sites/{SHAREPOINT_SITE_NAME}/{SHAREPOINT_DOC}/{folder_name}'
        target_folder = conn.web.get_folder_by_server_relative_path(target_folder_url)
        response = target_folder.upload_file(file_name, content).execute_query()
        return response
    
    def upload_file_in_chunks(self, file_path, folder_name, chunk_size, chunk_uploaded=None, **kwargs):
        conn = self._auth()
        target_folder_url = f'/sites/{SHAREPOINT_SITE_NAME}/{SHAREPOINT_DOC}/{folder_name}'
        target_folder = conn.web.get_folder_by_server_relative_path(target_folder_url)
        response = target_folder.files.create_upload_session(
            source_path=file_path,
            chunk_size=chunk_size,
            chunk_uploaded=chunk_uploaded,
            **kwargs
        ).execute_query()
        return response
    
    def get_list(self, list_name):
        conn = self._auth()
        target_list = conn.web.lists.get_by_title(list_name)
        items = target_list.items.get().execute_query()
        return items
        
    def get_file_properties_from_folder(self, folder_name):
        files_list = self._get_files_list(folder_name)
        properties_list = []
        for file in files_list:
            file_dict = {
                'file_id': file.unique_id,
                'file_name': file.name,
                'major_version': file.major_version,
                'minor_version': file.minor_version,
                'file_size': file.length,
                'time_created': file.time_created,
                'time_last_modified': file.time_last_modified
            }
            properties_list.append(file_dict)
            file_dict = {}
        return properties_list

SyntaxError: invalid syntax (environ.py, line 114)

In [8]:
from office365_api import SharePoint
import re
import sys, os
from pathlib import PurePath

# 1 args = Root Directory Path of files to upload
ROOT_DIR = sys.argv[1]
# 2 args = SharePoint folder name. May include subfolders to upload to
SHAREPOINT_FOLDER_NAME = sys.argv[2]
# 3 args = File name pattern. Only upload files with this pattern
FILE_NAME_PATTERN = sys.argv[3]


def upload_files(folder, keyword=None):
    file_list = get_list_of_files(folder)
    for file in file_list:
        if keyword is None or keyword == 'None' or re.search(keyword, file[0]):
            file_content = get_file_content(file[1])
            SharePoint().upload_file(file[0], SHAREPOINT_FOLDER_NAME, file_content)

def get_list_of_files(folder):
    file_list = []
    folder_item_list = os.listdir(folder)
    for item in folder_item_list:
        item_full_path = PurePath(folder, item)
        if os.path.isfile(item_full_path):
            file_list.append([item, item_full_path])
    return file_list

# read files and return the content of files
def get_file_content(file_path):
    with open(file_path, 'rb') as f:
        return f.read()

if __name__ == '__main__':
    upload_files(ROOT_DIR, FILE_NAME_PATTERN)


ModuleNotFoundError: No module named 'office365_api'

#### Nguyên phụ liệu

In [3]:
dfProductUnit = pd.read_csv('../dataDA/ProductUnit.csv')
dfProductUnit

,product_id,product_unit_id,unit_id,name_unit,is_default
0,21,21,1,Mét,1
1,22,22,1,Mét,1
2,26,26,1,Mét,1
3,28,28,1,Mét,1
4,31,31,1,Mét,1
...,...,...,...,...,...
16842,7909,7919,20,Pia,1
16843,7914,7924,21,Cây,1
16844,7916,7926,21,Cây,1
16845,7917,7927,21,Cây,1


In [4]:
dfProductUnitDefault = dfProductUnit[dfProductUnit['is_default'] == 1]
dfProductUnitDefault

,product_id,product_unit_id,unit_id,name_unit,is_default
0,21,21,1,Mét,1
1,22,22,1,Mét,1
2,26,26,1,Mét,1
3,28,28,1,Mét,1
4,31,31,1,Mét,1
...,...,...,...,...,...
16842,7909,7919,20,Pia,1
16843,7914,7924,21,Cây,1
16844,7916,7926,21,Cây,1
16845,7917,7927,21,Cây,1


In [118]:
dfOrder = pd.read_csv('../dataDA/Product.csv')
dfOrder

,name,product_id,is_outsourcing,product_code,quantity_product
0,Lưới màn 80G Black,1,0,1-LUOIMA-BLK-80-CM,4256.70
1,Lưới màn 80G White,2,0,1-LUOIMA-WHI-80-CM,7659.29
2,Lưới màn 80G Light White,3,0,1-LUOIMA-LWH-80-CM,4437.70
3,Lưới màn mỏng 28G Black,4,0,1-LUOIMA-BLK-28-CM,1128.50
4,Lưới màn mỏng 28G White,5,0,1-LUOIMA-WHI-28-CM,2367.60
...,...,...,...,...,...
11228,Thùng 3 lớp 34*31*25cm màu vàng không in,11543,0,9-343125-Y00-3L-BN,0.00
11229,Thùng 5 lớp 70*37*37cm màu vàng có in SA,11544,0,9-703737-Y01-5L-BN,0.00
11230,Thùng 5 lớp 66*39*55cm màu vàng có in SA,11545,0,9-663955-Y01-5L-BN,0.00
11231,Tấm lót 5 lớp 33*39cm,11546,0,9-TL3339-Y00-5L-BN,NaN


In [121]:
dfProduct = pd.read_csv('../dataDA/Product.csv')
dfProductUnit = dfProduct.merge(dfProductUnitDefault , on = 'product_id' , how = 'left').drop(columns= {'product_unit_id' , 'unit_id' , 'is_default'})
dfProductUnit

,name,product_id,is_outsourcing,product_code,quantity_product,name_unit
0,Lưới màn 80G Black,1,0,1-LUOIMA-BLK-80-CM,4256.70,Mét
1,Lưới màn 80G White,2,0,1-LUOIMA-WHI-80-CM,7659.29,Mét
2,Lưới màn 80G Light White,3,0,1-LUOIMA-LWH-80-CM,4437.70,Mét
3,Lưới màn mỏng 28G Black,4,0,1-LUOIMA-BLK-28-CM,1128.50,Mét
4,Lưới màn mỏng 28G White,5,0,1-LUOIMA-WHI-28-CM,2367.60,Mét
...,...,...,...,...,...,...
11228,Thùng 3 lớp 34*31*25cm màu vàng không in,11543,0,9-343125-Y00-3L-BN,0.00,Cái
11229,Thùng 5 lớp 70*37*37cm màu vàng có in SA,11544,0,9-703737-Y01-5L-BN,0.00,Cái
11230,Thùng 5 lớp 66*39*55cm màu vàng có in SA,11545,0,9-663955-Y01-5L-BN,0.00,Cái
11231,Tấm lót 5 lớp 33*39cm,11546,0,9-TL3339-Y00-5L-BN,NaN,Cái


In [122]:
dfRelationship = pd.read_csv('../dataDA/Relationship.csv')
dfRelationship

,product_id,outsourcing_product_id,product_code,quota_outsourcing
0,73,75,1-CVAC04-SCA-00-IW,1.0
1,73,78,1-CVAC04-SCA-00-IW,1.0
2,73,79,1-CVAC04-SCA-00-IW,1.0
3,80,81,1-CVAC05-DBN-00-IW,1.0
4,96,101,1-CVAC07-GRY-00-HV,1.0
...,...,...,...,...
4803,1334,7773,1-R81050-BLK-L2-EV,0.0
4804,1343,7785,1-R81050-STL-L1-EV,0.0
4805,10646,7783,1-R81050-WTR-L1-EV,0.0
4806,9736,8864,1-P2585Q-PLL-00-EV,1.0


In [124]:
dfProductUnit.to_excel('../dataDA/ProductUnit{}.xlsx'.format(now), index= False)

#### Production order + Stock Out

In [88]:
dfProductionOrder = pd.read_csv('../dataDA/ProductionOrderDetailBao.csv')
dfProductionOrder

,product_code,order_id,production_order_id,production_code,quantity_production_order,quantity_exported_production_order,time_create,completed_at,time_stock_out
0,4-CVAC06-BLK-A0-35,1,17,PYC-07-THU-DUC,309.00,45.00,2022-05-13,2022-05-16,2022-06-14
1,7-CMCUOC-WHI-00-AK,1,17,PYC-07-THU-DUC,1.00,1.00,2022-05-13,2022-05-16,2022-05-25
2,9-X44317-Y00-5L-BN,1,114,PYC-16-THUDUC,5.00,5.00,2022-05-30,2022-05-31,2022-06-04
3,3-NH9454-BLK-23-CY,1,17,PYC-07-THU-DUC,500.00,500.00,2022-05-13,2022-05-16,2022-05-25
4,9-HCHOAM-000-00-NP,1,17,PYC-07-THU-DUC,20.00,20.00,2022-05-13,2022-05-16,2022-05-25
...,...,...,...,...,...,...,...,...,...
45039,7-CM3673-PNK-50-PP,2650,2713,PYC-412-THIENDUC,5.00,5.00,2022-12-28,2022-12-29,2022-12-28
45040,1-THUNME-PEW-L7-CM,2650,2720,PYC-1050-CAT,14.28,14.28,2022-12-28,2022-12-29,2022-12-28
45041,4-KATE00-WHI-N0-28,2650,2713,PYC-412-THIENDUC,105.00,105.00,2022-12-28,2022-12-29,2022-12-28
45042,7-CM9631-GRY-50-PP,2650,2713,PYC-412-THIENDUC,1.00,1.00,2022-12-28,2022-12-29,2022-12-28


In [89]:
dfOrder = pd.read_csv('../dataDA/OrderBao.csv')

In [90]:
dfProductionOrder = dfProductionOrder.groupby(['product_code', 'order_id' ,'production_order_id' , 'production_code',
                                    'quantity_production_order' ,
                                    'time_create' , 'completed_at' , 'time_stock_out'], as_index=False)['quantity_exported_production_order'].sum()

dfProductionOrder = dfProductionOrder.groupby(['product_code', 'order_id' ,'production_order_id' , 'production_code',
                                    'time_create' , 'completed_at' , 'time_stock_out' , 'quantity_exported_production_order'], as_index=False)['quantity_production_order'].sum()

dfProductionOrder


,product_code,order_id,production_order_id,production_code,time_create,completed_at,time_stock_out,quantity_exported_production_order,quantity_production_order
0,1-1035CW-ACG-00-UL,1398,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,329.95,329.95
1,1-1035CW-ACG-00-UL,1399,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,410.02,410.02
2,1-1035CW-ACG-L1-UL,1398,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,41.47,41.47
3,1-1035CW-ACG-L1-UL,1399,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,47.08,47.08
4,1-1035CW-SGY-00-UL,1400,1839,PYC-667-CAT,2022-10-18,2022-10-19,2022-10-18,110.48,110.48
...,...,...,...,...,...,...,...,...,...
41425,9-X44317-Y00-5L-BN,1913,1637,PYC-215-THUDUC,2022-10-03,2022-10-04,2022-10-03,1.00,1.00
41426,9-X44317-Y00-5L-BN,1963,1713,PYC-227-THUDUC,2022-10-07,2022-10-08,2022-10-08,0.50,0.50
41427,9-X44317-Y00-5L-BN,1964,1713,PYC-227-THUDUC,2022-10-07,2022-10-08,2022-10-08,0.50,0.50
41428,9-X44317-Y00-5L-BN,2516,2519,PYC-374-THIENDUC,2022-12-12,2022-12-13,2022-12-14,1.51,1.51


In [91]:
dfOrder = dfOrder.groupby(['product_id', 'order_code' ,'order_id' ,
                                    'product_code'], as_index=False)['total_quota'].sum()

In [92]:
dfProductionOrder = dfProductionOrder.replace(np.NaN , 0)
dfProductionOrderMerge = pd.merge(dfOrder, dfProductionOrder , on = ['product_code' , 'order_id'], how = 'left')
dfProductionOrderMerge = dfProductionOrderMerge.replace(np.NaN , 0)

In [93]:
dfProductionOrderMerge[(dfProductionOrderMerge['order_code'] == 'AD-220623-YELIR 2')].head(2)

,product_id,order_code,order_id,product_code,total_quota,production_order_id,production_code,time_create,completed_at,time_stock_out,quantity_exported_production_order,quantity_production_order
360,82,AD-220623-YELIR 2,763,1-CVAC06-BLK-00-HV,96.0,499.0,PYC-194-CAT,2022-06-30,2022-07-01,2022-07-01,95.999999,96.0
361,82,AD-220623-YELIR 2,763,1-CVAC06-BLK-00-HV,96.0,499.0,PYC-194-CAT,2022-06-30,2022-07-01,2022-07-04,0.000001,96.0


In [94]:
# dfProductionOrderMerge[dfProductionOrderMerge['order_code'] == 'Charter Hat-Med-Cream-Sand']

In [95]:
dfProductionOrderMerge = dfProductionOrderMerge.drop(columns= {'product_id' , 'order_id' ,'production_order_id'})
dfProductionOrderMerge.rename(columns = {'order_code':'Mã đơn hàng' , 'product_code': 'Mã NPL' , 'total_quota': 'Định mức đơn hàng' , 'production_code': 'Mã LSX' , 'quantity_production_order' : 'SL LSX' , 
                                         'quantity_exported_production_order' : 'Tổng đã xuất' , 'time_create': 'Ngày tạo LSX',
                                         'completed_at': 'Ngày hẹn LSX', 'time_stock_out': 'ngày xuất kho'}, inplace = True)
dfProductionOrderMerge

,Mã đơn hàng,Mã NPL,Định mức đơn hàng,Mã LSX,Ngày tạo LSX,Ngày hẹn LSX,ngày xuất kho,Tổng đã xuất,SL LSX
0,CZ68-220615-Recycle Plastic Visor Blue Navy NE...,1-LUOIMA-BLK-80-CM,432.91,PYC-528-CAT,2022-09-21,2022-09-22,2022-09-21,432.91,432.91
1,CZ68-220615-Recycle Plastic Visor DK-Green MI,1-LUOIMA-BLK-80-CM,113.90,PYC-281-CAT,2022-07-26,2022-07-27,2022-07-26,113.90,113.90
2,CZ68-220615-Recycle Plastic Visor DK-Green NEW...,1-LUOIMA-BLK-80-CM,371.11,PYC-528-CAT,2022-09-21,2022-09-22,2022-09-21,300.00,300.00
3,DYC-AU-220304-Ashco Energy-Opt02-v2,1-LUOIMA-BLK-80-CM,8.84,0,0,0,0,0.00,0.00
4,DYC-AU-220906-Grand Banks-Opt04,1-LUOIMA-BLK-80-CM,30.86,0,0,0,0,0.00,0.00
...,...,...,...,...,...,...,...,...,...
52710,AD-221209-REDFILLS IR SIGNATURE,4-KTUYEN-BLK-N0-36,96.78,PYC-962-PHOI,2022-12-28,2022-12-29,2022-12-28,96.78,96.78
52711,AD-221216-REDFILLS MABRE,4-KTUYEN-BLK-N0-36,96.78,PYC-962-PHOI,2022-12-28,2022-12-29,2022-12-28,96.78,96.78
52712,AD-221216-REDFILLS MABRE GOLD,4-KTUYEN-BLK-N0-36,96.78,PYC-962-PHOI,2022-12-28,2022-12-29,2022-12-28,96.78,96.78
52713,AD-221216-REDFILLS MABRE GRIS,4-KTUYEN-BLK-N0-36,96.78,PYC-962-PHOI,2022-12-28,2022-12-29,2022-12-28,96.78,96.78


In [96]:
dfProductionOrderMerge[dfProductionOrderMerge['Mã NPL'] == '9-CCI166-000-00-TA']

,Mã đơn hàng,Mã NPL,Định mức đơn hàng,Mã LSX,Ngày tạo LSX,Ngày hẹn LSX,ngày xuất kho,Tổng đã xuất,SL LSX
37136,CIE-221218-Soleil BK Hat SM,9-CCI166-000-00-TA,246.0,0,0,0,0,0.0,0.0


In [79]:
dfProductionOrderMerge.to_excel('../dataDA/getProductionOrder{}.xlsx'.format(now), index= False)

#### Outsourcing Stock In


In [39]:
dfProduct = pd.read_csv("../dataDA/Product.csv")

In [40]:
dfPrimary = dfProduct.loc[dfProduct['is_outsourcing'] == 0]
dfSecondary = dfProduct.loc[dfProduct['is_outsourcing'] == 1]

In [41]:
dfPrimary

,product_id,is_outsourcing,product_code,quantity_product
0,1,0,1-LUOIMA-BLK-80-CM,4368.70
1,2,0,1-LUOIMA-WHI-80-CM,7774.29
2,3,0,1-LUOIMA-LWH-80-CM,4437.70
3,4,0,1-LUOIMA-BLK-28-CM,1623.50
4,5,0,1-LUOIMA-WHI-28-CM,2744.60
...,...,...,...,...
11217,11532,0,7-CTG815-000-50-NI,12.00
11218,11533,0,1-TAFETA-BLK-00-CA,0.00
11219,11534,0,1-TAFETA-KHA-00-CA,0.00
11220,11535,0,9-GBC006-WHI-00-MH,0.00


In [42]:
dfProduct

,product_id,is_outsourcing,product_code,quantity_product
0,1,0,1-LUOIMA-BLK-80-CM,4368.70
1,2,0,1-LUOIMA-WHI-80-CM,7774.29
2,3,0,1-LUOIMA-LWH-80-CM,4437.70
3,4,0,1-LUOIMA-BLK-28-CM,1623.50
4,5,0,1-LUOIMA-WHI-28-CM,2744.60
...,...,...,...,...
11217,11532,0,7-CTG815-000-50-NI,12.00
11218,11533,0,1-TAFETA-BLK-00-CA,0.00
11219,11534,0,1-TAFETA-KHA-00-CA,0.00
11220,11535,0,9-GBC006-WHI-00-MH,0.00


In [43]:
dfOutsourcingStockIn = pd.read_csv('../dataDA/OutsourcingStockInBao.csv')
dfOutSourcingDetail = pd.read_csv('../dataDA/OutSourcingDetailBao.csv')

In [44]:
dfOutSourcingDetail[dfOutSourcingDetail['outsourcing_order_id'] == 138].shape

(30, 9)

In [45]:
dfOutsourcingStockIn[dfOutsourcingStockIn['outsourcing_order_id'] == 138].shape

(30, 5)

In [46]:
dfOutSourcingDetail.sort_values(by = ['outsourcing_order_id'])

,code,name,product_id,outsourcing_order_id,is_complete_order,created_at_,delivery_at_,outsourcing_quantity,is_complete_order_detail
5280,16052022-2,DYC-CH1-220507-James Blunt,8422,1,1,2022-05-18,2022-05-19,159.0,1
5279,18052022-3,AD-220305-REDFILLS PATCH & PATTERN 2,8367,2,1,2022-05-18,2022-05-19,124.0,0
5278,18052022-4,BF-220402-PEMBROKE CLASSIC SPORT GREY,8356,3,1,2022-05-18,2022-05-19,126.0,0
5277,18052022-5,AD-220305-REDFILLS PATCH & PATTERN 2 BF-220330...,3983,4,1,2022-05-18,2022-05-19,5000.0,0
5276,18052022-6,BF-220402-PEMBROKE CLASSIC SPORT NAVY 282C,8418,5,1,2022-05-18,2022-05-19,63.0,1
...,...,...,...,...,...,...,...,...,...
4,29122022-1584,TTD-281222-276-Trung Việt,8441,1583,0,2022-12-29,NaN,105.0,0
3,29122022-1584,TTD-281222-276-Trung Việt,8436,1583,0,2022-12-29,NaN,89.0,0
2,29122022-1584,TTD-281222-276-Trung Việt,1197,1583,0,2022-12-29,NaN,5.0,0
1,29122022-1584,TTD-281222-276-Trung Việt,1202,1583,0,2022-12-29,NaN,15.0,0


In [47]:
dfOutsourcingStockIn

,product_id,outsourcing_order_id,outsourcing_quantity,quantity_stock_in,delivery_real_
0,11413,1584,330.0,NaN,NaN
1,1197,1583,5.0,NaN,NaN
2,8436,1583,89.0,NaN,NaN
3,8441,1583,105.0,NaN,NaN
4,2337,1583,12.5,NaN,NaN
...,...,...,...,...,...
5995,8418,5,63.0,63.0,2022-05-25
5996,3983,4,5000.0,NaN,NaN
5997,8356,3,126.0,113.0,2022-05-25
5998,8367,2,124.0,NaN,NaN


In [48]:
dfOutsourcingStockIn[dfOutsourcingStockIn['outsourcing_order_id'] == 295]

,product_id,outsourcing_order_id,outsourcing_quantity,quantity_stock_in,delivery_real_
4946,9028,295,392.0,392.0,2022-07-11
4947,9028,295,80.0,80.0,2022-07-11
4948,9028,295,80.0,392.0,2022-07-11
4949,9028,295,392.0,80.0,2022-07-11


In [49]:
dfOutSourcingDetail

,code,name,product_id,outsourcing_order_id,is_complete_order,created_at_,delivery_at_,outsourcing_quantity,is_complete_order_detail
0,29122022-1585,TTDKTY-221221-VS520-Chateau Gray-Blank,11413,1584,0,2022-12-29,2022-12-30,330.0,0
1,29122022-1584,TTD-281222-276-Trung Việt,1202,1583,0,2022-12-29,NaN,15.0,0
2,29122022-1584,TTD-281222-276-Trung Việt,1197,1583,0,2022-12-29,NaN,5.0,0
3,29122022-1584,TTD-281222-276-Trung Việt,8436,1583,0,2022-12-29,NaN,89.0,0
4,29122022-1584,TTD-281222-276-Trung Việt,8441,1583,0,2022-12-29,NaN,105.0,0
...,...,...,...,...,...,...,...,...,...
5276,18052022-6,BF-220402-PEMBROKE CLASSIC SPORT NAVY 282C,8418,5,1,2022-05-18,2022-05-19,63.0,1
5277,18052022-5,AD-220305-REDFILLS PATCH & PATTERN 2 BF-220330...,3983,4,1,2022-05-18,2022-05-19,5000.0,0
5278,18052022-4,BF-220402-PEMBROKE CLASSIC SPORT GREY,8356,3,1,2022-05-18,2022-05-19,126.0,0
5279,18052022-3,AD-220305-REDFILLS PATCH & PATTERN 2,8367,2,1,2022-05-18,2022-05-19,124.0,0


In [50]:
dfOutSourcingDetailMerge = dfOutSourcingDetail.merge(dfOutsourcingStockIn , on = ['outsourcing_order_id' , 
                                                            'product_id' , 'outsourcing_quantity'], how = 'right').replace(np.NaN , 0)
dfOutSourcingDetailMerge

,code,name,product_id,outsourcing_order_id,is_complete_order,created_at_,delivery_at_,outsourcing_quantity,is_complete_order_detail,quantity_stock_in,delivery_real_
0,29122022-1585,TTDKTY-221221-VS520-Chateau Gray-Blank,11413,1584,0,2022-12-29,2022-12-30,330.0,0,0.0,0
1,29122022-1584,TTD-281222-276-Trung Việt,1197,1583,0,2022-12-29,0,5.0,0,0.0,0
2,29122022-1584,TTD-281222-276-Trung Việt,8436,1583,0,2022-12-29,0,89.0,0,0.0,0
3,29122022-1584,TTD-281222-276-Trung Việt,8441,1583,0,2022-12-29,0,105.0,0,0.0,0
4,29122022-1584,TTD-281222-276-Trung Việt,2337,1583,0,2022-12-29,0,12.5,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
6016,18052022-6,BF-220402-PEMBROKE CLASSIC SPORT NAVY 282C,8418,5,1,2022-05-18,2022-05-19,63.0,1,63.0,2022-05-25
6017,18052022-5,AD-220305-REDFILLS PATCH & PATTERN 2 BF-220330...,3983,4,1,2022-05-18,2022-05-19,5000.0,0,0.0,0
6018,18052022-4,BF-220402-PEMBROKE CLASSIC SPORT GREY,8356,3,1,2022-05-18,2022-05-19,126.0,0,113.0,2022-05-25
6019,18052022-3,AD-220305-REDFILLS PATCH & PATTERN 2,8367,2,1,2022-05-18,2022-05-19,124.0,0,0.0,0


In [51]:
dfProduct[dfProduct['product_id'] == 8617]

,product_id,is_outsourcing,product_code,quantity_product


In [52]:
dfOutSourcingDetailMerge = dfOutSourcingDetailMerge.merge(dfProduct[['product_id' , 'product_code']] , on = 'product_id'  , how = 'left')
# dfOutSourcingDetailMerge

In [53]:
# dfOutSourcingDetailMerge[dfOutSourcingDetailMerge['outsourcing_order_id'] == 138]

In [54]:
dfOutSourcingDetailMerge = dfOutSourcingDetailMerge.drop(columns= {'product_id' , 'outsourcing_order_id'})
dfOutSourcingDetailMerge.rename(columns = {'code':'Mã đơn hàng' , 'name': 'Tên đơn hàng', 'product_code': 'Mã NPL' , 'is_complete_order': 'Hoàn tất đơn hàng' ,
                                         'is_complete_order_detail': 'Hoàn tất chi tiết đơn hàng' , 'outsourcing_quantity': 'Số lượng GCN' , 'quantity_stock_in' : 'Số lượng GCN nhập kho' , 
                                        'created_at_': 'Ngày tạo ĐH', 'delivery_at_': 'Ngày hẹn ĐH', 'delivery_real_': 'Ngày xuất kho GCN'}, inplace = True)
dfOutSourcingDetailMerge

,Mã đơn hàng,Tên đơn hàng,Hoàn tất đơn hàng,Ngày tạo ĐH,Ngày hẹn ĐH,Số lượng GCN,Hoàn tất chi tiết đơn hàng,Số lượng GCN nhập kho,Ngày xuất kho GCN,Mã NPL
0,29122022-1585,TTDKTY-221221-VS520-Chateau Gray-Blank,0,2022-12-29,2022-12-30,330.0,0,0.0,0,4-P2585Q-CTG-L3-32
1,29122022-1584,TTD-281222-276-Trung Việt,0,2022-12-29,0,5.0,0,0.0,0,1-P983UV-RNY-L1-EV
2,29122022-1584,TTD-281222-276-Trung Việt,0,2022-12-29,0,89.0,0,0.0,0,1-HW1RIN-WHI-L3-HF
3,29122022-1584,TTD-281222-276-Trung Việt,0,2022-12-29,0,105.0,0,0.0,0,1-HW1RIN-WHI-L1-HF
4,29122022-1584,TTD-281222-276-Trung Việt,0,2022-12-29,0,12.5,0,0.0,0,1-THUPIN-WHI-M5-AT
...,...,...,...,...,...,...,...,...,...,...
6016,18052022-6,BF-220402-PEMBROKE CLASSIC SPORT NAVY 282C,1,2022-05-18,2022-05-19,63.0,1,63.0,2022-05-25,4-CVAC14-NAV-D6-30
6017,18052022-5,AD-220305-REDFILLS PATCH & PATTERN 2 BF-220330...,1,2022-05-18,2022-05-19,5000.0,0,0.0,0,5-DAFELT-BLK-07-TA
6018,18052022-4,BF-220402-PEMBROKE CLASSIC SPORT GREY,1,2022-05-18,2022-05-19,126.0,0,113.0,2022-05-25,4-CVAC07-GRY-D5-30
6019,18052022-3,AD-220305-REDFILLS PATCH & PATTERN 2,1,2022-05-18,2022-05-19,124.0,0,0.0,0,4-SUEDE0-BLK-D8-30


In [55]:
dfOutSourcingDetailMerge.to_excel('../dataDA/getOutSourcingStockIn{}.xlsx'.format(now), index= False)

In [56]:
dfOutSourcingDetailMerge[dfOutSourcingDetailMerge['Hoàn tất chi tiết đơn hàng'].isnull()]

,Mã đơn hàng,Tên đơn hàng,Hoàn tất đơn hàng,Ngày tạo ĐH,Ngày hẹn ĐH,Số lượng GCN,Hoàn tất chi tiết đơn hàng,Số lượng GCN nhập kho,Ngày xuất kho GCN,Mã NPL


#### Outsourcing Stock Out

In [57]:
dfOutSourcingMaterial = pd.read_csv('../dataDA/OutSourcingMaterialBao.csv')
dfOutSourcingMaterial = dfOutSourcingMaterial.replace(np.NaN , 0)
dfOutSourcingMaterial

,product_id,outsourcing_detail_id,outsourcing_product_id,outsourcing_order_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent
0,8311,5,8422,1,159.0,1,0.069,0,0.0000
1,235,5,8422,1,159.0,1,0.069,0,0.0000
2,7923,5,8422,1,159.0,1,0.069,0,0.0000
3,1499,7,8367,2,124.0,0,8.000,0,0.0000
4,8311,7,8367,2,124.0,0,8.000,0,0.0000
...,...,...,...,...,...,...,...,...,...
7365,8434,6782,8436,1583,89.0,0,89.000,1,1.0000
7366,2334,6784,2337,1583,12.5,0,12.500,1,1.0000
7367,2,6780,1202,1583,15.0,0,15.000,1,1.0000
7368,5,6782,8436,1583,89.0,0,89.000,1,1.0000


In [58]:
dfProductNew  = dfProduct.rename(columns={"product_id": "outsourcing_product_id" , "product_code" : "outsourcing_product_code"})
dfOutSourcingMaterialMerge = dfOutSourcingMaterial.merge(dfProduct[['product_id' , 
                    'product_code']] , on = 'product_id' , how = 'left').merge(dfProductNew[['outsourcing_product_id' , 
                    'outsourcing_product_code']] , on = 'outsourcing_product_id' , how = 'left')
dfOutSourcingMaterialMerge

,product_id,outsourcing_detail_id,outsourcing_product_id,outsourcing_order_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent,product_code,outsourcing_product_code
0,8311,5,8422,1,159.0,1,0.069,0,0.0000,1-GOLDAI-WHI-00-EP,4-CVAC57-OLI-D6-30
1,235,5,8422,1,159.0,1,0.069,0,0.0000,1-CVAC57-OLI-00-HV,4-CVAC57-OLI-D6-30
2,7923,5,8422,1,159.0,1,0.069,0,0.0000,9-GIA80G-BLK-00-TQ,4-CVAC57-OLI-D6-30
3,1499,7,8367,2,124.0,0,8.000,0,0.0000,1-PH563B-TIT-00-UL,4-SUEDE0-BLK-D8-30
4,8311,7,8367,2,124.0,0,8.000,0,0.0000,1-GOLDAI-WHI-00-EP,4-SUEDE0-BLK-D8-30
...,...,...,...,...,...,...,...,...,...,...,...
7365,8434,6782,8436,1583,89.0,0,89.000,1,1.0000,1-HW1RIN-WHI-00-HF,1-HW1RIN-WHI-L3-HF
7366,2334,6784,2337,1583,12.5,0,12.500,1,1.0000,1-THUPIN-WHI-00-AT,1-THUPIN-WHI-M5-AT
7367,2,6780,1202,1583,15.0,0,15.000,1,1.0000,1-LUOIMA-WHI-80-CM,1-P983UV-SHL-L1-EV
7368,5,6782,8436,1583,89.0,0,89.000,1,1.0000,1-LUOIMA-WHI-28-CM,1-HW1RIN-WHI-L3-HF


In [59]:
dfOutSourcingStockOut = pd.read_csv('../dataDA/OutSourcingStockOutBao.csv')
dfOutSourcingStockOut = dfOutSourcingStockOut.replace(np.NaN , 0)
dfOutSourcingStockOut

,product_id,outsourcing_order_id,is_exported,outsourcing_detail_id,assign_product_stockout_id,quantity_exported_outsourcing,date_of_inventory
0,1,358,0,1751,12027,27.5,2022-07-12
1,1,641,1,2918,24364,92.0,2022-08-22
2,1,1441,1,6279,52521,117.0,2022-12-09
3,1,368,0,1791,12321,10.5,2022-07-13
4,1,1500,1,6449,54640,51.0,2022-12-17
...,...,...,...,...,...,...,...
6395,11520,1538,1,6623,56597,2.0,2022-12-23
6396,11521,1538,1,6628,56607,1.5,2022-12-23
6397,11522,1538,1,6625,56604,1.5,2022-12-23
6398,11523,1538,1,6624,56603,2.0,2022-12-23


In [60]:
dfOutSourcingStockOut = dfOutSourcingStockOut.groupby(['product_id' , 'outsourcing_order_id' , 'is_exported',
                                                       'outsourcing_detail_id', 'date_of_inventory' ], as_index=False)['quantity_exported_outsourcing'].sum()
dfOutSourcingStockOut.sort_values(by = ['product_id'])
dfOutSourcingStockOut

,product_id,outsourcing_order_id,is_exported,outsourcing_detail_id,date_of_inventory,quantity_exported_outsourcing
0,1,50,0,330,2022-05-25,17.0
1,1,50,0,331,2022-05-25,9.0
2,1,50,0,333,2022-05-25,16.0
3,1,50,0,334,2022-05-25,8.0
4,1,50,0,336,2022-05-25,21.0
...,...,...,...,...,...,...
5933,11520,1538,1,6623,2022-12-23,2.0
5934,11521,1538,1,6628,2022-12-23,1.5
5935,11522,1538,1,6625,2022-12-23,1.5
5936,11523,1538,1,6624,2022-12-23,2.0


In [61]:
dfOutSourcingMaterialMergeNew = dfOutSourcingMaterialMerge.merge(dfOutSourcingStockOut, on = ['product_id', 'outsourcing_detail_id' , 'outsourcing_order_id'
                                                        , 'is_exported'] , how = 'left')

dfOutSourcingMaterialMergeNew

,product_id,outsourcing_detail_id,outsourcing_product_id,outsourcing_order_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent,product_code,outsourcing_product_code,date_of_inventory,quantity_exported_outsourcing
0,8311,5,8422,1,159.0,1,0.069,0,0.0000,1-GOLDAI-WHI-00-EP,4-CVAC57-OLI-D6-30,NaN,NaN
1,235,5,8422,1,159.0,1,0.069,0,0.0000,1-CVAC57-OLI-00-HV,4-CVAC57-OLI-D6-30,NaN,NaN
2,7923,5,8422,1,159.0,1,0.069,0,0.0000,9-GIA80G-BLK-00-TQ,4-CVAC57-OLI-D6-30,NaN,NaN
3,1499,7,8367,2,124.0,0,8.000,0,0.0000,1-PH563B-TIT-00-UL,4-SUEDE0-BLK-D8-30,NaN,NaN
4,8311,7,8367,2,124.0,0,8.000,0,0.0000,1-GOLDAI-WHI-00-EP,4-SUEDE0-BLK-D8-30,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7382,8434,6782,8436,1583,89.0,0,89.000,1,1.0000,1-HW1RIN-WHI-00-HF,1-HW1RIN-WHI-L3-HF,2022-12-29,89.00
7383,2334,6784,2337,1583,12.5,0,12.500,1,1.0000,1-THUPIN-WHI-00-AT,1-THUPIN-WHI-M5-AT,2022-12-29,12.50
7384,2,6780,1202,1583,15.0,0,15.000,1,1.0000,1-LUOIMA-WHI-80-CM,1-P983UV-SHL-L1-EV,2022-12-29,15.00
7385,5,6782,8436,1583,89.0,0,89.000,1,1.0000,1-LUOIMA-WHI-28-CM,1-HW1RIN-WHI-L3-HF,2022-12-29,89.00


In [62]:
dfOutSourcingMaterialMergeNew[dfOutSourcingMaterialMergeNew['outsourcing_order_id'] == 1375]

,product_id,outsourcing_detail_id,outsourcing_product_id,outsourcing_order_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent,product_code,outsourcing_product_code,date_of_inventory,quantity_exported_outsourcing
6501,2170,6079,2171,1375,233.5,1,233.5,1,1.0,1-WOOACR-CAL-00-KJ,1-WOOACR-CAL-L2-KJ,2022-12-02,233.5
6502,1,6079,2171,1375,233.5,1,233.5,1,1.0,1-LUOIMA-BLK-80-CM,1-WOOACR-CAL-L2-KJ,2022-12-02,233.5
6503,2272,6077,2275,1375,19.5,1,19.5,1,1.0,1-THUNP0-BLK-00-AT,1-THUNP0-BLK-M2-AT,2022-12-02,21.0
6504,102,6080,105,1375,6.5,1,6.5,1,1.0,1-CVAC08-WHI-00-HV,1-CVAC08-WHI-L1-HV,2022-12-02,6.5
6505,2272,6078,2276,1375,26.0,1,26.0,1,1.0,1-THUNP0-BLK-00-AT,1-THUNP0-BLK-M5-AT,2022-12-02,27.0
6506,2,6080,105,1375,6.5,1,6.5,1,1.0,1-LUOIMA-WHI-80-CM,1-CVAC08-WHI-L1-HV,2022-12-02,6.5


In [63]:
dfOutSourcingOrder = pd.read_csv('../dataDA/OutSourcingOrderBao.csv')
dfOutSourcingOrder

,code,name,outsourcing_order_id,is_complete_infos
0,29122022-1585,TTDKTY-221221-VS520-Chateau Gray-Blank,1584,0
1,29122022-1584,TTD-281222-276-Trung Việt,1583,0
2,29122022-1583,TTDKTY-221221,1582,0
3,29122022-1582,TTD-291222-100-Hằng Phong,1581,0
4,29122022-1581,DYC-US-221219-E11even-677-FVC-1017,1580,0
...,...,...,...,...
1564,18052022-6,BF-220402-PEMBROKE CLASSIC SPORT NAVY 282C,5,1
1565,18052022-5,AD-220305-REDFILLS PATCH & PATTERN 2 BF-220330...,4,1
1566,18052022-4,BF-220402-PEMBROKE CLASSIC SPORT GREY,3,1
1567,18052022-3,AD-220305-REDFILLS PATCH & PATTERN 2,2,1


In [64]:
dfOutSourcingMaterialMerge = dfOutSourcingMaterialMergeNew.merge(dfOutSourcingOrder, on = ['outsourcing_order_id'] , how = 'left')
dfOutSourcingMaterialMerge.head(2)

,product_id,outsourcing_detail_id,outsourcing_product_id,outsourcing_order_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent,product_code,outsourcing_product_code,date_of_inventory,quantity_exported_outsourcing,code,name,is_complete_infos
0,8311,5,8422,1,159.0,1,0.069,0,0.0,1-GOLDAI-WHI-00-EP,4-CVAC57-OLI-D6-30,NaN,NaN,16052022-2,DYC-CH1-220507-James Blunt,1
1,235,5,8422,1,159.0,1,0.069,0,0.0,1-CVAC57-OLI-00-HV,4-CVAC57-OLI-D6-30,NaN,NaN,16052022-2,DYC-CH1-220507-James Blunt,1


In [65]:
dfRelationShip = pd.read_csv('../dataDA/Relationship.csv')
dfRelationShipOrder = dfRelationShip.rename(columns= {'product_code' : 'product_code_po'})
dfRelationShipOrder

,product_id,outsourcing_product_id,product_code_po,quota_outsourcing
0,73,75,1-CVAC04-SCA-00-IW,1.0
1,73,78,1-CVAC04-SCA-00-IW,1.0
2,73,79,1-CVAC04-SCA-00-IW,1.0
3,80,81,1-CVAC05-DBN-00-IW,1.0
4,96,101,1-CVAC07-GRY-00-HV,1.0
...,...,...,...,...
4826,9458,10404,9-GIA80G-BLK-40-TA,1.0
4827,7770,10404,9-PH563B-BLK-00-95,1.0
4828,1334,7773,1-R81050-BLK-L2-EV,0.0
4829,1343,7785,1-R81050-STL-L1-EV,0.0


In [66]:
dfOutSourcingMaterialRel = dfOutSourcingMaterialMerge.merge(dfRelationShipOrder , on = ['outsourcing_product_id' , 'product_id'] , how = 'left')
dfOutSourcingMaterialRel.head(2)

,product_id,outsourcing_detail_id,outsourcing_product_id,outsourcing_order_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent,product_code,outsourcing_product_code,date_of_inventory,quantity_exported_outsourcing,code,name,is_complete_infos,product_code_po,quota_outsourcing
0,8311,5,8422,1,159.0,1,0.069,0,0.0,1-GOLDAI-WHI-00-EP,4-CVAC57-OLI-D6-30,NaN,NaN,16052022-2,DYC-CH1-220507-James Blunt,1,NaN,NaN
1,235,5,8422,1,159.0,1,0.069,0,0.0,1-CVAC57-OLI-00-HV,4-CVAC57-OLI-D6-30,NaN,NaN,16052022-2,DYC-CH1-220507-James Blunt,1,NaN,NaN


In [67]:
dfOutSourcingMaterialRel = dfOutSourcingMaterialRel.drop(columns= {'product_id' , 'outsourcing_order_id' , 'outsourcing_product_id' , 'outsourcing_order_id' , 'outsourcing_detail_id'})

new_col1 = ['code','name' , 'product_code' ,'outsourcing_product_code', 'total_quota', 
      'quantity_outsourcing_detail', 'is_complete_detail', 'is_exported',
      'is_complete_infos', 'quota_percent' , 'quota_outsourcing' , 'date_of_inventory',
      'quantity_exported_outsourcing' , 'product_code_po']
   
dfOutSourcingMaterialStockOut= dfOutSourcingMaterialRel.reindex(columns=new_col1)
   
dfOutSourcingMaterialStockOut.rename(columns = {'code':'Mã ĐH GCN' , 'name': 'Tên ĐH GCN', 'product_code': 'Mã WIP' , 'outsourcing_product_code': 'Mã Thành phẩm',
                                             'total_quota': 'SL WIP cần GCN', 'quantity_outsourcing_detail': 'SL thành phẩm cần', 'is_complete_detail': 'Nhập kho Thành phẩm',
                                        'is_exported': 'Đã Xuất WIP' , 'is_complete_infos': 'ĐH hoàn tất' , 'quota_percent': 'Định mức GCN' , 'date_of_inventory': 'Ngày xuất kho',
                                        'quantity_exported_outsourcing':'Số lượng WIP đi GCN thực tế' , 'quota_outsourcing' : 'Định mức theo thu mua' , 'product_code_po' : 'Mã NPL bán TP theo thu mua'}, inplace = True)
dfOutSourcingMaterialStockOut

,Mã ĐH GCN,Tên ĐH GCN,Mã WIP,Mã Thành phẩm,SL WIP cần GCN,SL thành phẩm cần,Nhập kho Thành phẩm,Đã Xuất WIP,ĐH hoàn tất,Định mức GCN,Định mức theo thu mua,Ngày xuất kho,Số lượng WIP đi GCN thực tế,Mã NPL bán TP theo thu mua
0,16052022-2,DYC-CH1-220507-James Blunt,1-GOLDAI-WHI-00-EP,4-CVAC57-OLI-D6-30,0.069,159.0,1,0,1,0.0000,NaN,NaN,NaN,NaN
1,16052022-2,DYC-CH1-220507-James Blunt,1-CVAC57-OLI-00-HV,4-CVAC57-OLI-D6-30,0.069,159.0,1,0,1,0.0000,NaN,NaN,NaN,NaN
2,16052022-2,DYC-CH1-220507-James Blunt,9-GIA80G-BLK-00-TQ,4-CVAC57-OLI-D6-30,0.069,159.0,1,0,1,0.0000,NaN,NaN,NaN,NaN
3,18052022-3,AD-220305-REDFILLS PATCH & PATTERN 2,1-PH563B-TIT-00-UL,4-SUEDE0-BLK-D8-30,8.000,124.0,0,0,1,0.0000,NaN,NaN,NaN,NaN
4,18052022-3,AD-220305-REDFILLS PATCH & PATTERN 2,1-GOLDAI-WHI-00-EP,4-SUEDE0-BLK-D8-30,8.000,124.0,0,0,1,0.0000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7382,29122022-1584,TTD-281222-276-Trung Việt,1-HW1RIN-WHI-00-HF,1-HW1RIN-WHI-L3-HF,89.000,89.0,0,1,0,1.0000,1.0000,2022-12-29,89.00,1-HW1RIN-WHI-00-HF
7383,29122022-1584,TTD-281222-276-Trung Việt,1-THUPIN-WHI-00-AT,1-THUPIN-WHI-M5-AT,12.500,12.5,0,1,0,1.0000,1.0000,2022-12-29,12.50,1-THUPIN-WHI-00-AT
7384,29122022-1584,TTD-281222-276-Trung Việt,1-LUOIMA-WHI-80-CM,1-P983UV-SHL-L1-EV,15.000,15.0,0,1,0,1.0000,1.0000,2022-12-29,15.00,1-LUOIMA-WHI-80-CM
7385,29122022-1584,TTD-281222-276-Trung Việt,1-LUOIMA-WHI-28-CM,1-HW1RIN-WHI-L3-HF,89.000,89.0,0,1,0,1.0000,1.0000,2022-12-29,89.00,1-LUOIMA-WHI-28-CM


In [68]:
dfOutSourcingMaterialStockOut.to_excel('../dataDA/getOutSourcingStockOut{}.xlsx'.format(now), index= False)

#### PYC nhap kho